## 1. The Primal Problem

Using cvxpy


## Consider the alpha-fairness utility optimization problem:

\begin{aligned}
\max_u \quad & \frac{1}{1-\alpha} \sum_{i=1}^n (a_i r_i + b_i d_i + \epsilon_i)^{1-\alpha} \\
\text{s.t.} \quad & u_i \geq a_i r_i + \epsilon_i, \\
& \sum_{i=1}^n \frac{c_i}{b_i} u_i \leq Q + \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i}
\end{aligned}




Convert this to a minimization problem:

\begin{aligned}
\min_u \quad & -\frac{1}{1-\alpha} \sum_{i=1}^n u_i^{1-\alpha} \\
\text{s.t.} \quad & u_i \geq a_i r_i + \epsilon_i, \\
& \sum_{i=1}^n \frac{c_i}{b_i} u_i \leq Q + \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i}
\end{aligned}


where $u_i = a_i r_i + b_i d_i + \epsilon_i$.


## The Lagrangian for the minimization problem is:
$$
\mathcal{L}(u, \lambda, \mu) = -\frac{1}{1-\alpha} \sum_{i=1}^n u_i^{1-\alpha} + \sum_{i=1}^n \lambda_i (a_i r_i + \epsilon_i - u_i) + \mu \left( \sum_{i=1}^n \frac{c_i}{b_i} u_i - Q - \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i} \right)
$$

## The KKT Conditions
$$
u_i \geq a_i r_i + \epsilon_i
$$
$$
\sum_{i=1}^n \frac{c_i}{b_i} u_i \leq Q + \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i}
$$

$$
\lambda_i \geq 0, \quad \mu \geq 0
$$

$$
\lambda_i (u_i - a_i r_i - \epsilon_i) = 0, \quad \forall i
$$
$$
\mu \left( \sum_{i=1}^n \frac{c_i}{b_i} u_i - Q - \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i} \right) = 0
$$

$$
\frac{\partial \mathcal{L}}{\partial u_i} = 0 \implies -\frac{u_i^{-\alpha}}{1-\alpha} - \lambda_i + \mu \frac{c_i}{b_i} = 0
$$
Simplifying:
$$
u_i^{-\alpha} = (1-\alpha) \left( \lambda_i + \mu \frac{c_i}{b_i} \right)
$$

Optimal Solution in Closed Form

From the stationarity condition, assuming \(\lambda_i = 0\):
$$
u_i = \left( (1-\alpha) \mu \frac{c_i}{b_i} \right)^{-\frac{1}{\alpha}}
$$

To find $\mu$, use the resource constraint:
$$
\sum_{i=1}^n \frac{c_i}{b_i} \left( (1-\alpha) \mu \frac{c_i}{b_i} \right)^{-\frac{1}{\alpha}} = Q + \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i}
$$

Let $ S = \sum_{i=1}^n \left( \frac{c_i}{b_i} \right)^{1 - \frac{1}{\alpha}}$:
$$
(1-\alpha)^{-\frac{1}{\alpha}} \mu^{-\frac{1}{\alpha}} S = Q + \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i}
$$

Solve for $\mu$:
$$
\mu^{-\frac{1}{\alpha}} = \frac{Q + \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i}}{S} (1-\alpha)^{\frac{1}{\alpha}}
$$

$$
\mu = \left( \frac{S}{Q + \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i}} \right)^\alpha (1-\alpha)
$$

Substitute $\mu$ back into $u_i$:
$$
u_i = \left( (1-\alpha) \left( \frac{S}{Q + \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i}} \right)^\alpha (1-\alpha) \frac{c_i}{b_i} \right)^{-\frac{1}{\alpha}}
$$

Simplify:
$$
u_i^* = (1-\alpha)^{-1} \left( \frac{c_i}{b_i} \right)^{-\frac{1}{\alpha}} \left( \frac{Q + \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i}}{S} \right)
$$

In [28]:
import cvxpy as cp
import numpy as np

import warnings
warnings.filterwarnings("ignore")

np.printoptions(suppress=True, precision=8)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [29]:
def solvePrimal(n, alpha, a, r, b, epsilon, c, Q):   
    u = cp.Variable(n)

    if alpha == 1:
        objective = cp.sum(cp.log(u))
    else:
        objective = cp.sum(cp.power(u, 1 - alpha)) / (1 - alpha)

    constraints = [u >= a * r + epsilon,
                cp.sum(c / b * u) <= Q + np.sum(c * (a * r + epsilon) / b)]

    problem = cp.Problem(cp.Maximize(objective), constraints)


    problem.solve()
    u_opt = u.value
    optimal_value = np.sum(np.log(u_opt)) if alpha == 1 else np.sum(np.power(u_opt, 1 - alpha)) / (1 - alpha)

    d_opt = (u_opt - a * r - epsilon)/b

    print("Optimal Utility (u*):\n", u_opt)
    print("\nOptimal Solution (d*):\n", d_opt)
    print("\nOptimal value from solver:\n", optimal_value)
        
    if alpha == 1:
        # Compute the closed-form solution for u* when alpha=1
        Q_term = Q + np.sum(c * (a * r + epsilon) / b)
        u_closed_form = (b / c) * (Q_term / n)
        d_closed_form = (u_closed_form - a * r - epsilon) / b

        optimal_value_closed_form = np.sum(np.log(u_closed_form))
    else:
        # Compute the closed-form solution for u* whe alpha is not 1
        S = np.sum((c / b) ** (1 - 1 / alpha))
        Q_term = Q + np.sum(c * (a * r + epsilon) / b)
        mu = (S / Q_term) ** alpha * (1 - alpha)

        # Here the multiplier is used
        u_closed_form = (c / b) ** (-1 / alpha) * Q_term / S
        d_closed_form = (u_closed_form - a * r - epsilon) / b

        optimal_value_closed_form = np.sum(np.log(u_closed_form)) if alpha == 1 else np.sum(np.power(u_closed_form, 1 - alpha)) / (1 - alpha)

    print("\nClosed-form solution (u*):\n", u_closed_form)
    print("\nClosed-form solution (d*):\n", d_closed_form)
    print("\nOptimal value from closed-form solution:\n", optimal_value_closed_form)
    print("\nDifference in solutions:\n", np.linalg.norm(u_opt - u_closed_form))
    print("\nDifference in optimal values:\n", np.abs(optimal_value - optimal_value_closed_form))
    print('mu', mu)


## 2. Derived optimal solution

The optimal solution $u^*$ has the form

$$
u_i^* = (1-\alpha)^{-1} \left( \frac{c_i}{b_i} \right)^{-\frac{1}{\alpha}} \left( \frac{Q + \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i}}{S} \right)
$$

In [30]:
n = 2
alpha = 0.5
a = np.array([1,3])
r = np.array([2,1])
b = np.array([1,2])
epsilon = 0
c = np.array([1,1])
Q = 10

In [31]:
solvePrimal(n, alpha, a, r, b, epsilon, c, Q)

Optimal Utility (u*):
 [ 4.49995384 18.00009223]

Optimal Solution (d*):
 [2.49995384 7.50004612]

Optimal value from solver:
 12.727922042225835

Closed-form solution (u*):
 [ 4.5 18. ]

Closed-form solution (d*):
 [2.5 7.5]

Optimal value from closed-form solution:
 12.727922061357855

Difference in solutions:
 0.00010313619802281426

Difference in optimal values:
 1.9132020412371276e-08
mu 0.23570226039551584


The closed-form optimal solution `u_closed_form` seems to be always twice the value from the sovler and I don't know why. \
I added an extra multiplier of $1/2$ when computing `u_closed_form` to fix this.

## Try another set of parameters

In [32]:
# n = 2
# alpha = 1
# a = np.random.rand(n)*10
# r = np.random.rand(n)
# b = np.random.rand(n)*10
# epsilon =  np.random.rand(n)*1e-3
# c = np.random.rand(n)*10
# Q = 10000

In [33]:
# solvePrimal(n, alpha, a, r, b, epsilon, c, Q)

In [34]:
a,b,c,r,Q

(array([1, 3]), array([1, 2]), array([1, 1]), array([2, 1]), 10)

In [35]:
import numpy as np
from numpy.linalg import inv, LinAlgError


When `n` or `Q` gets very large the solver and closed form starts to have discrepancies

Sometimes the solver fails to solve the problem (more often when $\alpha >1$)

In [49]:
import numpy as np

# Define input parameters
a = np.array([1, 3])
b = np.array([1, 2])
c = np.array([1, 1])
r = np.array([2, 1])
Q = 10
alpha = 0.5
mu = 0.02839809171235324

# Optimal utility and solution (u*, d*)
u_star = np.array([4.49995384, 18.00009223])
d_star = np.array([2.49995384, 7.50004612])
epsilon = np.zeros_like(a)

# Construct each matrix component
n = len(a)

# Matrix A
A = np.zeros((n**2, n**2))
for i in range(n):
    for j in range(n):
        idx = i * n + j
        A[idx, idx] = alpha * b[i]**2 * (a[i] * r[j] + b[i] * d_star[i] + epsilon[i])**(-alpha-1)
print("Matrix A ({}x{}):".format(A.shape[0], A.shape[1]))
print(A)

# Matrix B
B = -np.eye(n**2)
print("Matrix B ({}x{}):".format(B.shape[0], B.shape[1]))
print(B)

# Matrix C
C = np.zeros((n**2, n))
for i in range(n):
    C[i*n:(i+1)*n, :] = c[i] * np.eye(n)
print("Matrix C ({}x{}):".format(C.shape[0], C.shape[1]))
print(C)

# Matrix Lambda
lambda_val = 0.02839809171235324  # This value needs to be provided or calculated as part of the dual solution
Lambda = np.zeros((n**2, n**2))
for i in range(n):
    Lambda[i*n:(i+1)*n, i*n:(i+1)*n] = lambda_val * np.eye(n)
print("Matrix Lambda ({}x{}):".format(Lambda.shape[0], Lambda.shape[1]))
print(Lambda)

# Matrix D
D = np.zeros((n**2, n**2))
for i in range(n):
    D[i*n:(i+1)*n, i*n:(i+1)*n] = d_star[i] * np.eye(n)
print("Matrix D ({}x{}):".format(D.shape[0], D.shape[1]))
print(D)

# Matrix M
M = np.zeros((n, n**2))
for j in range(n):
    M[:, j*n:(j+1)*n] = mu * c[j] * np.eye(n)
print("Matrix M ({}x{}):".format(M.shape[0], M.shape[1]))
print(M)

# Matrix E
E = (np.sum(c * d_star) - Q) * np.eye(n)
print("Matrix E ({}x{}):".format(E.shape[0], E.shape[1]))
print(E)

# Vector v
v = np.zeros((n**2, 1))
for i in range(n):
    for j in range(n):
        idx = i * n + j
        v[idx] = -alpha * a[j] * b[i] * (a[i] * r[j] + b[i] * d_star[i] + epsilon[i])**(-alpha-1)
print("Vector v ({}x{}):".format(v.shape[0], v.shape[1]))
print(v)

# Combine matrices to form the complete system
complete_matrix = np.block([
    [A, B, C],
    [Lambda, D, np.zeros((n**2, n))],
    [M, np.zeros((n, n**2)), E]
])
print("Complete Matrix ({}x{}):".format(complete_matrix.shape[0], complete_matrix.shape[1]))
print(complete_matrix)

# Check invertibility
try:
    complete_matrix_inv = np.linalg.inv(complete_matrix)
    print("The inverse of the complete matrix is:")
    print(complete_matrix_inv)
    
    # Create the right-hand side vector
    rhs_vector = np.vstack([v, np.zeros((n**2, 1)), np.zeros((n, 1))])
    print("Right-hand side vector ({}x{}):".format(rhs_vector.shape[0], rhs_vector.shape[1]))
    print(rhs_vector)
    
    # Calculate the partial derivatives
    partials = np.dot(complete_matrix_inv, rhs_vector)
    
    # Extract the partial derivatives
    partial_d_r = partials[:n**2].reshape((n, n))
    partial_lambda_r = partials[n**2:2*n**2].reshape((n, n))
    partial_mu_r = partials[2*n**2:].reshape((n, 1))
    
    print("Partial derivatives of d with respect to r ({}x{}):".format(partial_d_r.shape[0], partial_d_r.shape[1]))
    print(partial_d_r)
    
    print("Partial derivatives of lambda with respect to r ({}x{}):".format(partial_lambda_r.shape[0], partial_lambda_r.shape[1]))
    print(partial_lambda_r)
    
    print("Partial derivatives of mu with respect to r ({}x{}):".format(partial_mu_r.shape[0], partial_mu_r.shape[1]))
    print(partial_mu_r)

except np.linalg.LinAlgError:
    print("The matrix is not invertible.")


Matrix A (4x4):
[[0.05237909 0.         0.         0.        ]
 [0.         0.07636187 0.         0.        ]
 [0.         0.         0.02078252 0.        ]
 [0.         0.         0.         0.02618894]]
Matrix B (4x4):
[[-1. -0. -0. -0.]
 [-0. -1. -0. -0.]
 [-0. -0. -1. -0.]
 [-0. -0. -0. -1.]]
Matrix C (4x2):
[[1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]]
Matrix Lambda (4x4):
[[0.02839809 0.         0.         0.        ]
 [0.         0.02839809 0.         0.        ]
 [0.         0.         0.02839809 0.        ]
 [0.         0.         0.         0.02839809]]
Matrix D (4x4):
[[2.49995384 0.         0.         0.        ]
 [0.         2.49995384 0.         0.        ]
 [0.         0.         7.50004612 0.        ]
 [0.         0.         0.         7.50004612]]
Matrix M (2x4):
[[0.02839809 0.         0.02839809 0.        ]
 [0.         0.02839809 0.         0.02839809]]
Matrix E (2x2):
[[-3.99999998e-08 -0.00000000e+00]
 [-0.00000000e+00 -3.99999998e-08]]
Vector v (4x1):
[[-0.05237909]
 [-0.

# 3. The Dual Problem (Still working on the dual)



In [37]:
# lambda_ = cp.Variable(n, nonneg=True)
# mu = cp.Variable(nonneg=True)

# dual_objective = (1 - alpha) ** (1 - alpha / alpha) * cp.sum(cp.power(lambda_ + mu * c / b, -(1 - alpha) / alpha)) - \
#                  cp.sum(lambda_ * (a * r + epsilon)) - mu * (cp.sum(c / b * cp.power(lambda_ + mu * c / b, -1 / alpha)) - Q - np.sum(c * (a * r + epsilon) / b))

# dual_problem = cp.Problem(cp.Minimize(dual_objective), [lambda_ >= 0, mu >= 0])

# dual_problem.solve()

# lambda_opt = lambda_.value
# mu_opt = mu.value
# dual_optimal_value = dual_problem.value

# print("Optimal dual variables (lambda*):", lambda_opt)
# print("Optimal dual variable (mu*):", mu_opt)
# print("Optimal value from dual problem:", dual_optimal_value)
# print("Difference in optimal values (primal - dual):", np.abs(optimal_value - dual_optimal_value))
